In [7]:
from selenium import webdriver
import pandas as pd
import re
import time
import os
from functools import reduce

In [4]:
def seletor(driver,identificador,texto):
    # seleciona o ID
    grupo_id = driver.find_element_by_id(identificador)
    select = grupo_id.find_element_by_class_name("multiselect__select")
    select.click()

    lista_opcoes = grupo_id.find_element_by_class_name("multiselect__content")\
                            .find_elements_by_class_name("multiselect__option")
    for l in lista_opcoes:
        if l.text == texto:
            l.click()
            
def configuracao(driver,ano,mes,regiao = "Sudeste",estado = "São Paulo"):
    #Ano
    seletor(driver,"datePickrGroup",ano)
    #Meses
    seletor(driver,"__BVID__74",mes)
    #Região
    seletor(driver,"__BVID__78",regiao)
    #Estado
    seletor(driver,"__BVID__82",estado)

    #Pesquisar
    driver.find_elements_by_class_name("mt-4")[1].find_element_by_class_name("d-flex").click()
    
def recupera_tabela(driver):
    #Recupera o card_tabela e a tabela em sim
    card_tabela = driver.find_elements_by_class_name("card")[4]
    time.sleep(1)
    tabela = driver.find_element_by_class_name("table-responsive")

    #Percorre a tabela e insere as informações na lista
    lista = []
    lista_dados = tabela.find_element_by_tag_name("tbody").find_elements_by_tag_name("tr")
    for l in lista_dados:
        search = re.search("(.*)( )([0-9]+)",l.text)
        lista.append([search.group(1),search.group(3)])

    #Recupera a lista para avançar
    lista_avancar = card_tabela.find_element_by_tag_name("ul")
    
    return (lista,lista_avancar)

#Faça esse procedimento de recuperação dos dados da tabela até que não exista mais possibilidade de avançar
def export_backup(driver,mes,ano):
    mortes = []
    while True:
        lista,la = recupera_tabela(driver)
        mortes = mortes + lista
        atualmente_ativo = int(la.find_element_by_class_name("active").text)
        print("Mês: %s, Ano: %s, Pagina atual: %s"%(mes,ano,atualmente_ativo))
        if atualmente_ativo + 1 in [int(n) for n in la.text.split("\n") if n.isnumeric()]:
            proximo = atualmente_ativo + 1
        else:
            print("Fim exportação das tabelas\n\n")
            break

        li_list = la.find_elements_by_tag_name("li")
        for li in li_list:
            if li.text == str(proximo):
                li.click()
        #backup será exportado com sucesso
        if("obitos_sp_%s_%s_%s.csv"%(mes,ano,atualmente_ativo) not in os.listdir(path_backup)):
            pd.DataFrame.from_records(lista,columns=["Cidade","Registros"])\
                        .assign(Mes = mes, Ano = ano).to_csv(path_backup+"obitos_sp_%s_%s_%s.csv"%(mes,
                                                                                                   ano,
                                                                                                   atualmente_ativo),
                                                             index = False)
        else:
            print("Já exportado*")
    
    return True

In [5]:
driver = webdriver.Chrome("../../Covid-19/driver/chromedriver99")
driver.get("https://transparencia.registrocivil.org.br/registros")
path_backup = "../data/raw/Historico_obitos/Transparencia/"

combinacoes = [["2020","Abril"],
               ["2020","Maio"],
               ["2020","Junho"],
               ["2020","Julho"],
               ["2020","Agosto"],
               ["2020","Setembro"],
               ["2020","Outubro"],
               ["2020","Novembro"],
               ["2020","Dezembro"],
               ["2021","Janeiro"],
               ["2021","Fevereiro"],
               ["2021","Março"],
               ["2021","Abril"]]

for ano,mes in combinacoes:
    #Configuracoes iniciais
    print("-----------------------------------------------------")
    print("Configurações iniciais:%s/%s"%(mes,ano))
    configuracao(driver,ano,mes)
    print("Fim das configurações iniciais\n")
    not_fim = False
    while not_fim == False:
        try:
            not_fim = export_backup(driver,mes,ano)
        except:
            print("!!!!!!!!!!!!ERROR!!!!!!!!!!!!!")
            not_fim = export_backup(driver,mes,ano)

-----------------------------------------------------
Configurações iniciais:Abril/2020
Fim das configurações iniciais

Mês: Abril, Ano: 2020, Pagina atual: 1
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 2
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 3
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 4
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 5
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 6
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 7
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 8
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 9
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 10
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 11
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 12
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 13
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 14
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 15
Já exportado*
Mês: Abril, Ano: 2020, Pagina atual: 16
Já exportado*
Mês: Abril, Ano: 2020, Pa

Mês: Agosto, Ano: 2020, Pagina atual: 24
Mês: Agosto, Ano: 2020, Pagina atual: 25
Mês: Agosto, Ano: 2020, Pagina atual: 26
Mês: Agosto, Ano: 2020, Pagina atual: 27
Mês: Agosto, Ano: 2020, Pagina atual: 28
Mês: Agosto, Ano: 2020, Pagina atual: 29
Mês: Agosto, Ano: 2020, Pagina atual: 30
!!!!!!!!!!!!ERROR!!!!!!!!!!!!!
Mês: Agosto, Ano: 2020, Pagina atual: 31
Mês: Agosto, Ano: 2020, Pagina atual: 32
Mês: Agosto, Ano: 2020, Pagina atual: 33
Fim exportação das tabelas


-----------------------------------------------------
Configurações iniciais:Setembro/2020
Fim das configurações iniciais

Mês: Setembro, Ano: 2020, Pagina atual: 1
Mês: Setembro, Ano: 2020, Pagina atual: 2
Mês: Setembro, Ano: 2020, Pagina atual: 3
Mês: Setembro, Ano: 2020, Pagina atual: 4
Mês: Setembro, Ano: 2020, Pagina atual: 5
Mês: Setembro, Ano: 2020, Pagina atual: 6
Mês: Setembro, Ano: 2020, Pagina atual: 7
Mês: Setembro, Ano: 2020, Pagina atual: 8
Mês: Setembro, Ano: 2020, Pagina atual: 9
Mês: Setembro, Ano: 2020, Pag

Mês: Janeiro, Ano: 2021, Pagina atual: 31
Mês: Janeiro, Ano: 2021, Pagina atual: 32
Mês: Janeiro, Ano: 2021, Pagina atual: 33
Fim exportação das tabelas


-----------------------------------------------------
Configurações iniciais:Fevereiro/2021
Fim das configurações iniciais

Mês: Fevereiro, Ano: 2021, Pagina atual: 1
Mês: Fevereiro, Ano: 2021, Pagina atual: 2
Mês: Fevereiro, Ano: 2021, Pagina atual: 3
Mês: Fevereiro, Ano: 2021, Pagina atual: 4
Mês: Fevereiro, Ano: 2021, Pagina atual: 5
Mês: Fevereiro, Ano: 2021, Pagina atual: 6
Mês: Fevereiro, Ano: 2021, Pagina atual: 7
Mês: Fevereiro, Ano: 2021, Pagina atual: 8
Mês: Fevereiro, Ano: 2021, Pagina atual: 9
Mês: Fevereiro, Ano: 2021, Pagina atual: 10
Mês: Fevereiro, Ano: 2021, Pagina atual: 11
Mês: Fevereiro, Ano: 2021, Pagina atual: 12
Mês: Fevereiro, Ano: 2021, Pagina atual: 13
Mês: Fevereiro, Ano: 2021, Pagina atual: 14
Mês: Fevereiro, Ano: 2021, Pagina atual: 15
Mês: Fevereiro, Ano: 2021, Pagina atual: 16
Mês: Fevereiro, Ano: 2021,

In [11]:
nome_arquivos_juntar = [pd.read_csv(path_backup+fn) for fn in os.listdir(path_backup) if ".csv" in fn]
reduce(lambda x,y: pd.concat((x,y)), nome_arquivos_juntar).to_excel(path_backup+"obitos_sp.xlsx")